<a href="https://colab.research.google.com/github/gogimandu1019/bigdata202406/blob/main/5_3_type2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 심장마비 확률이 높은사람?
- 성별, 나이, 혈압, 콜레스테롤, 공복혈당, 최대 심박수 등의 컬럼이 있음
- 평가: ROC-AUC, 정확도(Accuracy), F1 을 구하시오
- target : output (1:심장마비 확률 높음, 0:심장마비 확률 낮음)
- csv파일 생성 : 수험번호.csv (예시 아래 참조)
~~~
id,output
41,0.633
28,0.123
222,0.355
~~~



## 데이터 불러오기

In [1]:
import pandas as pd

train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

print(train.shape, test.shape)

(242, 15) (61, 14)


##EDA

In [3]:
train.head()

,id,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,48,53,0,2,128,216,0,0,115,0,0.0,2,0,0,1
1,124,39,0,2,94,199,0,1,179,0,0.0,2,0,2,1
2,129,74,0,1,120,269,0,0,121,1,0.2,2,1,2,1
3,78,52,1,1,128,205,1,1,184,0,0.0,2,0,2,1
4,151,71,0,0,112,149,0,1,125,0,1.6,1,0,2,1


In [4]:
test.head()

,id,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall
0,41,48,1,1,130,245,0,0,180,0,0.2,1,0,2
1,28,65,0,2,140,417,1,0,157,0,0.8,2,1,2
2,222,65,1,3,138,282,1,0,174,0,1.4,1,1,2
3,106,69,1,3,160,234,1,0,131,0,0.1,1,1,2
4,108,50,0,1,120,244,0,1,162,0,1.1,2,0,2


In [6]:
train['output'].value_counts()

output
1    126
0    116
Name: count, dtype: int64

In [9]:
train.isnull().sum()

id          0
age         0
sex         0
cp          0
trtbps      0
chol        0
fbs         0
restecg     0
thalachh    0
exng        0
oldpeak     0
slp         0
caa         0
thall       0
output      0
dtype: int64

In [11]:
test.isnull().sum()

id          0
age         0
sex         0
cp          0
trtbps      0
chol        0
fbs         0
restecg     0
thalachh    0
exng        0
oldpeak     0
slp         0
caa         0
thall       0
dtype: int64

In [13]:
train.describe()

,id,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
count,242.000000,242.000000,242.000000,242.000000,242.000000,242.000000,242.000000,242.00000,242.000000,242.000000,242.000000,242.000000,242.000000,242.000000,242.000000
mean,154.706612,54.516529,0.673554,0.913223,131.070248,246.933884,0.119835,0.53719,149.590909,0.347107,1.019835,1.380165,0.785124,2.347107,0.520661
std,86.328576,9.021513,0.469885,1.020897,17.716978,52.236414,0.325441,0.53183,21.763201,0.477037,1.182137,0.621285,1.052218,0.607160,0.500608
min,0.000000,34.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.00000,88.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,81.250000,47.250000,0.000000,0.000000,120.000000,212.000000,0.000000,0.00000,133.250000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,158.500000,55.000000,1.000000,1.000000,130.000000,243.000000,0.000000,1.00000,152.000000,0.000000,0.600000,1.000000,0.000000,2.000000,1.000000
75%,228.750000,61.000000,1.000000,2.000000,140.000000,274.000000,0.000000,1.00000,165.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,302.000000,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.00000,195.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [14]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242 entries, 0 to 241
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   id        242 non-null    int64  
 1   age       242 non-null    int64  
 2   sex       242 non-null    int64  
 3   cp        242 non-null    int64  
 4   trtbps    242 non-null    int64  
 5   chol      242 non-null    int64  
 6   fbs       242 non-null    int64  
 7   restecg   242 non-null    int64  
 8   thalachh  242 non-null    int64  
 9   exng      242 non-null    int64  
 10  oldpeak   242 non-null    float64
 11  slp       242 non-null    int64  
 12  caa       242 non-null    int64  
 13  thall     242 non-null    int64  
 14  output    242 non-null    int64  
dtypes: float64(1), int64(14)
memory usage: 28.5 KB


In [15]:
train.nunique()

id          242
age          40
sex           2
cp            4
trtbps       47
chol        139
fbs           2
restecg       3
thalachh     83
exng          2
oldpeak      38
slp           3
caa           5
thall         4
output        2
dtype: int64

## 데이터 전처리 & 피처엔지니어링

In [16]:
train = train.drop('id', axis = 1)
test_id = test.pop('id')

In [17]:
test.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall
0,48,1,1,130,245,0,0,180,0,0.2,1,0,2
1,65,0,2,140,417,1,0,157,0,0.8,2,1,2
2,65,1,3,138,282,1,0,174,0,1.4,1,1,2
3,69,1,3,160,234,1,0,131,0,0.1,1,1,2
4,50,0,1,120,244,0,1,162,0,1.1,2,0,2


## 검증 데이터 분리

In [18]:
from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(train.drop('output', axis = 1), train['output'], test_size= 0.15, random_state=2022)

In [19]:
X_tr.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall
43,53,0,0,130,264,0,0,143,0,0.4,1,0,2
32,55,0,0,128,205,0,2,130,1,2.0,1,1,3
190,61,1,3,134,234,0,1,145,0,2.6,1,2,2
211,38,1,2,138,175,0,1,173,0,0.0,2,4,2
234,64,1,3,170,227,0,0,155,0,0.6,1,0,3


## 모델 & 평가

In [27]:
#랜덤포레스트
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, f1_score, accuracy_score

rf = RandomForestClassifier(random_state = 2022, max_depth = 5, n_estimators = 400) #n_estimator: 트리갯수 기본값 100

rf.fit(X_tr, y_tr)
pred = rf.predict(X_val)
pred_proba = rf.predict_proba(X_val)

print(roc_auc_score(y_val, pred_proba[:, 1]))
print(f1_score(y_val, pred))
print(accuracy_score(y_val, pred))
#baseline
# 0.9301242236024845
# 0.7906976744186046
# 0.7567567567567568

#max_depth = 3~12, 보통 3,5,7 중 하나
#n_estimator = 100 200 400 800 1000 : 100~200 정도 / 근데 많으면 느려짐
#max_depth = 3
# 0.9378881987577639
# 0.8636363636363636
# 0.8378378378378378

#max_depth = 5
# 0.9409937888198757
# 0.8444444444444444
# 0.8108108108108109
#max_depth = 5, n_estimator = 200
# 0.9440993788819876
# 0.8444444444444444
# 0.8108108108108109
#max_depth = 5, n_estimator = 400
# 0.9409937888198757
# 0.8444444444444444
# 0.8108108108108109

#max_depth = 7
# 0.9347826086956521
# 0.8444444444444444
# 0.8108108108108109

0.9409937888198757
0.8444444444444444
0.8108108108108109


In [34]:
#xgboost
from xgboost import XGBClassifier
xgb = XGBClassifier(random_state = 2022, max_depth = 5, n_estimators = 400, learning_rate = 0.05) #maxdepth 기본 3 . n_estimators 올리면 learning_rate를 낮춰야 함

xgb.fit(X_tr, y_tr)
pred = xgb.predict(X_val)
pred_proba = xgb.predict_proba(X_val)

print(roc_auc_score(y_val, pred_proba[:, 1]))
print(f1_score(y_val, pred))
print(accuracy_score(y_val, pred))

#max_depth 3~12
#n_estimators 100 ~ 1000 , 얘 올릴거면 learning rate 0.1 ~ 0.01 사이에서 줄여주어야 함

#baseline
# 0.9192546583850931
# 0.8444444444444444
# 0.8108108108108109

#max_depth = 5
# 0.906832298136646
# 0.8181818181818182
# 0.7837837837837838

#max_depth = 7
# 0.9099378881987576
# 0.8181818181818182
# 0.7837837837837838

#max_depth = 5, n_estimators = 400, learning_rate = 0.05
# 0.9006211180124224
# 0.8444444444444444
# 0.8108108108108109

0.9006211180124224
0.8444444444444444
0.8108108108108109


In [36]:
pred_proba = xgb.predict_proba(test)

## 예측 및 csv 제출

In [35]:
test.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall
0,48,1,1,130,245,0,0,180,0,0.2,1,0,2
1,65,0,2,140,417,1,0,157,0,0.8,2,1,2
2,65,1,3,138,282,1,0,174,0,1.4,1,1,2
3,69,1,3,160,234,1,0,131,0,0.1,1,1,2
4,50,0,1,120,244,0,1,162,0,1.1,2,0,2


In [39]:
pd.DataFrame({
    'id': test_id,
    'output': pred_proba[:,1]
}).to_csv("00000.csv", index=False)

In [41]:
pd.read_csv("00000.csv")

,id,output
0,41,0.995112
1,28,0.969060
2,222,0.680690
3,106,0.274922
4,108,0.996831
...,...,...
56,6,0.984140
57,269,0.003593
58,2,0.998842
59,14,0.990590


##정리

In [43]:
y_test = pd.read_csv("y_test.csv")
print(roc_auc_score(y_test, pred_proba[:,1]))

0.8997668997668998
